# **연도별 데이터로 블록맵 그리기**
- 기준이 되는 '좌표' 표시된 지역 데이터 - 위도&경도
- 연평균 표시된 데이터 : 2010 ~ 2020년
- 데이터 합친 뒤 파일(csv)로 저장
- 블록맵 그리기

### **좌표 데이터 정제**
- 좌표(latitude, logitude) 데이터 정제

In [ ]:
import pandas as pd

In [ ]:
# def blockmap():
#     p = pd.read_csv('data/e_korea_region.csv', encoding='CP949', header=0, engine='python')
#     p = p.set_index('addr')
    
#     year = pd.read_csv('data/fin/year2020.csv', encoding='utf-8', header=0, engine='python')
#     year = year.set_index('address')
    
#     re = pd.merge(p, year, how='outer', left_index=True, right_index=True)
    
#     re = re.fillna(0)
    
#     re = re[['x','y','광역시도','행정구역', 'year_avg']]
#     print(re)
    
#     re.to_csv('data/blockmap/df_blockmap2020.csv', encoding='CP949', index=True)

In [ ]:
## 좌표 행정구역명 기준
p = pd.read_csv('data/e_korea_region.csv', encoding='CP949', header=0, engine='python')
p

In [ ]:
p = p.set_index('addr')
p.head()

#### **연도별 미세먼지 데이터랑 좌표 데이터 합친 뒤 저장**

In [ ]:
year = pd.read_csv('data/fin/year2017.csv', encoding='utf-8', header=0, engine='python')
year

In [ ]:
year = year.set_index('address')
year.head()

In [ ]:
re = pd.merge(p, year, how='outer', left_index=True, right_index=True)
re.head()

In [ ]:
## NaN to 0 : fillna
re = re.fillna(0)
re

In [ ]:
re = re[['x','y','광역시도','행정구역', 'year_avg']]
re

In [ ]:
## 연도별 라벨링 저장
re.to_csv('data/blockmap/df_blockmap2016.csv', encoding='CP949', index=True)

## **blockmap**

#### **표준행정구역명 수정 필요**
- 세종특별자치시 세종특별자치시 -> 세종특별자치시 세종시
- 충청북도 진천시 -> 충청북도 진천군

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import rcParams, style
style.use('ggplot')

from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family = font_name)

In [ ]:
# re.to_csv('data/fin/dt_blockmap.csv', encoding='CP949', index=True)

In [ ]:
#반복 함수화
def seq():
    df = pd.read_csv('data/blockmap/df_blockmap2020.csv', encoding='CP949', header=0, engine='python')
    print(type(df))
    df = df.rename(columns={'Unnamed: 0':'address'})
    print(type(df))
    df = df.set_index('address')
    print(df)

In [ ]:
seq()

In [ ]:
df = pd.read_csv('data/blockmap/df_blockmap2020.csv', encoding='CP949', header=0, engine='python')
df

In [ ]:
df=df.rename(columns={'Unnamed: 0':'address'})
df

In [ ]:
df = df.set_index('address')
df

In [ ]:
df.index.unique()

In [ ]:
#한국지도의 블록맵 경계선 좌표를 리스트로 생성 #공유폴더에서 파일 받아옴
#시도별로 블록라인(검은색 선) 그려서 구분
BORDER_LINES = [
    [(3, 2), (5, 2), (5, 3), (9, 3), (9, 1)], # 인천
    [(2, 5), (3, 5), (3, 4), (8, 4), (8, 7), (7, 7), (7, 9), (4, 9), (4, 7), (1, 7)], # 서울
    [(1, 6), (1, 9), (3, 9), (3, 10), (8, 10), (8, 9),
     (9, 9), (9, 8), (10, 8), (10, 5), (9, 5), (9, 3)], # 경기도
    [(9, 12), (9, 10), (8, 10)], # 강원도
    [(10, 5), (11, 5), (11, 4), (12, 4), (12, 5), (13, 5),
     (13, 4), (14, 4), (14, 2)], # 충청남도
    [(11, 5), (12, 5), (12, 6), (15, 6), (15, 7), (13, 7),
     (13, 8), (11, 8), (11, 9), (10, 9), (10, 8)], # 충청북도
    [(14, 4), (15, 4), (15, 6)], # 대전시
    [(14, 7), (14, 9), (13, 9), (13, 11), (13, 13)], # 경상북도
    [(14, 8), (16, 8), (16, 10), (15, 10),
     (15, 11), (14, 11), (14, 12), (13, 12)], # 대구시
    [(15, 11), (16, 11), (16, 13)], # 울산시
    [(17, 1), (17, 3), (18, 3), (18, 6), (15, 6)], # 전라북도
    [(19, 2), (19, 4), (21, 4), (21, 3), (22, 3), (22, 2), (19, 2)], # 광주시
    [(18, 5), (20, 5), (20, 6)], # 전라남도
    [(16, 9), (18, 9), (18, 8), (19, 8), (19, 9), (20, 9), (20, 10)], # 부산시
]

In [ ]:
# import numpay as np

#블록맵 그리는 함수
#blockedMap #targetData : count나 MC_ratio 중 하나일 것  #title : 이름  #color : 차트 컬러
def draw_blockMap(blockedMap, targetData, title, color):
    #값의 범위 설정 #count/MC_ratio의 제일 높은 거 - 제일 낮은 거
    whitelabelmin = (max(blockedMap[targetData]) - min(blockedMap[targetData])) * 0.85 + min(blockedMap[targetData])
    # whitelabelmin = ((max(125) - min(0)) * 0.25 + min(0))
    # print(max(blockedMap[targetData]), min(blockedMap[targetData]))
    # whitelabelmin = 125
    
    #차트의 색상 밝기에 따라 폰트 컬러 설정해주려고 
    datalabel = targetData #count/MC_ratio
    # vmin = min(blockedMap[targetData])
    # vmax = max(blockedMap[targetData])
    vmin = 0
    vmax = 100
    
    #좌표에 표시에 맵데이터를 만드는 것
    mapdata = blockedMap.pivot(index='y', columns='x', values=targetData)
    # print(mapdata)
    #np.ma.maked_where(조건(mapdata = 0)을 만족하면 0, 아니면 원래의 값-mapdata)
    masked_mapdata = np.ma.masked_where(np.isnan(mapdata), mapdata)
    
    plt.figure(figsize=(15, 21))
    # plt.title(title, fontsize=25)
    
    #pcolor : 컬러맵으로 그려줌
    plt.pcolor(masked_mapdata, vmin=vmin, vmax=vmax, cmap = color, edgecolor='#aaaaaa', linewidth=0.2)
    #>> 여기까지 블록맵 그림 그리기까지 완료
    
    #지역이름 표시 #인덱스값과 행값 가져와 넣을 것
    for idx, row in blockedMap.iterrows():
        annocolor = 'white' if row[targetData] > whitelabelmin else 'black'
        #광역시 구 이름이 겹치는 경우가 많아서 시 단위 이름다 같이 표시(중구, 서구)
        if row['광역시도'].endswith('시') and not row['광역시도'].startswith('세종'):
            dispname = '{}\n{}'.format(row['광역시도'][:2], row['행정구역'][:-1])
            #'광역시도'의 앞에서 두 글자, '행정구역'은 맨끝글자 제외
            if len(row['행정구역']) <= 2: #'행정구역' 글자가 2보다 작을 경우, 맨끝글자 붙임
                dispname += row['행정구역'][-1]
        else:
            dispname = row['행정구역'][:-1]
        
        #서대문구, 서귀포시 같이 이름이 3자이상인 경우 작은 글자로 표시
        #splitlines 나눔
        if len(dispname.splitlines()[-1]) >= 3:
            fontsize, linespacing = 9.5, 1.5
        else:
            fontsize, linespacing = 11, 1.2
        
        #ha(horizontal align), va(vertical align)
        plt.annotate(dispname, (row['x']+0.5, row['y']+0.5), weight='bold',
                               fontsize = fontsize, ha='center', va='center', color=annocolor,
                               linespacing = linespacing)
    #>> 블록맵 그리고 글자 입력까지 완료
    #시도 경계 그리기
    for path in BORDER_LINES:
        ys, xs = zip(*path)
        plt.plot(xs, ys, c='black', lw=4)
    
    #gca() : 축 객체 얻어옴
    #invert_yaxis() : 축을 뒤집음. 수학적 축 객체는 밑에서부터 시작하기 때문에
    plt.gca().invert_yaxis()
    #plt.gca().set_aspect(1)
    plt.axis('off')
    
    cb = plt.colorbar(shrink=.1, aspect=10)
    cb.set_label(datalabel)
    
    plt.tight_layout() #바깥여백을 타이트하게
    plt.savefig('data/blockmap/'+'blockMap_'+targetData+'2020.png') #그림 파일 저장
    plt.show()

In [ ]:
draw_blockMap(df, 'y', '미세먼지 평균', 'YlGn')  #PuBu  #YlGnBu